# Goal

<h3 style="color:blue">assess the quality of summaries written by students</h3>
<h3 style="color:indigo">evaluate how well a student represents the main idea and details of a source text, as well as the clarity, precision, and fluency of the language used in the summary</h3>
<h3 style="color:red">Freely & publicly available external data is <b>allowed</b>, including pre-trained models</h3>
<h3>This is Multi-Output problem</h3>

### Use Hugging Face Library
### Use NLTK
### Use Tensorflow

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import math
import subprocess
from tqdm import tqdm
import pickle

In [16]:
import tensorflow as tf

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, median_absolute_error

In [18]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [19]:
import transformers
from transformers import AutoTokenizer, TFBertModel

In [20]:
prompts_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

In [21]:
train = pd.merge(prompts_train, summaries_train, on='prompt_id')
test = pd.merge(prompts_test, summaries_test, on='prompt_id')

In [22]:
train.rename(columns = {'text' : 'summary'}, inplace=True)
test.rename(columns = {'text' : 'summary'}, inplace=True)

In [23]:
train.head(2)

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,summary,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058


In [24]:
columns_needed = ["prompt_text", "summary"]
train_data = train[columns_needed]
test_data = test[columns_needed]

In [25]:
from transformers import AutoTokenizer, TFAutoModel
model = TFAutoModel.from_pretrained('/kaggle/input/sentence-transformers/minilm-l6-v2/all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/sentence-transformers/minilm-l6-v2/all-MiniLM-L6-v2')

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at /kaggle/input/sentence-transformers/minilm-l6-v2/all-MiniLM-L6-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Next time use prepare_tf_dataset which is used to directly tokenize and data colat and
### make dataset compatible with tensorflow
####       https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset

In [26]:

def vectorize_dataframe(dataframe, col):
    vectors = []
    for text in tqdm(dataframe[col].tolist()):
        text_tokens = tokenizer(text, return_tensors="tf", padding=True, truncation=True)
        
        output = model(**text_tokens)
        array = np.array(output.pooler_output)
        vectors.append( array.flatten() )
        
    return vectors
    

In [27]:
train_data['prompt_text_embedded'] = vectorize_dataframe(train_data, 'prompt_text')

100%|██████████| 7165/7165 [12:17<00:00,  9.72it/s]


In [28]:
train_data['summary_embedded'] = vectorize_dataframe(train_data, 'summary')

100%|██████████| 7165/7165 [11:58<00:00,  9.97it/s]


In [29]:
test_data['prompt_text_embedded'] = vectorize_dataframe(test_data, 'prompt_text')
test_data['summary_embedded'] = vectorize_dataframe(test_data, 'summary')

100%|██████████| 4/4 [00:00<00:00, 10.17it/s]


In [30]:
#with open("/kaggle/working/sentence_transformer_prompt_text_embeddings.pkl", "wb") as file:
    #pickle.dump(train_data['prompt_text_embedded'], file)
    
#with open("/kaggle/working/sentence_transformer_summary_embeddings.pkl", "wb") as file:
 #   pickle.dump(train_data['summary_embedded'], file)
    
#with open("/kaggle/input/embeddings/BERT_summary_embeddings.pkl", "rb") as file:
 #   train_data['summary_embedded'] = pickle.load(file)

In [31]:
traning_set = train_data[['prompt_text_embedded', 'summary_embedded']]
testing_set = test_data[['prompt_text_embedded', 'summary_embedded']]

### Take average of embeddings  [Not required, just checking]

In [32]:
target1 = np.array(train['content'])
target1 = target1.astype('float32')

target2 = np.array(train['wording'])
target2 = target2.astype('float32')

#target = (target1, target2)

In [33]:
def convert_tensor_to_numpy(tensor):
        return np.array(tensor, dtype='float32')

traning_set = traning_set.applymap(convert_tensor_to_numpy)
testing_set = testing_set.applymap(convert_tensor_to_numpy)

In [34]:
def prepare_dataset(dataset):
    # Flatten the nested arrays in the DataFrame
    dataset['prompt_text_embedded'] = dataset['prompt_text_embedded'].apply(lambda x: x.flatten())
    dataset['summary_embedded'] = dataset['summary_embedded'].apply(lambda x: x.flatten())
    
    feature1 = np.array(dataset['prompt_text_embedded'].tolist())
    feature2 = np.array(dataset['summary_embedded'].tolist())
    
    features = np.concatenate((feature1, feature2), axis=1)
    
    return features

In [35]:
features = prepare_dataset(traning_set)

In [36]:
features_for_test = prepare_dataset(testing_set)

In [37]:
#X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [38]:
from tensorflow.keras.layers import Dense, Input, Flatten

In [39]:
# Define the model architecture
model_content = tf.keras.Sequential([
      tf.keras.layers.Dense(356, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(232, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(94, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1, activation='linear')
])

model_wording = tf.keras.Sequential([
     tf.keras.layers.Dense(356, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(232, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(94, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(1, activation='linear')
])

In [40]:
model_content.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
model_wording.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [41]:
# Train your model using model.fit()
history1 = model_content.fit(features, target1, epochs=200, batch_size=25, validation_split=0.2)

Epoch 1/200
230/230 [==============================] - 4s 5ms/step - loss: 1.0225 - mae: 0.7931 - mse: 1.0225 - val_loss: 0.7622 - val_mae: 0.6994 - val_mse: 0.7622
Epoch 2/200
230/230 [==============================] - 1s 4ms/step - loss: 0.6688 - mae: 0.6376 - mse: 0.6688 - val_loss: 0.4975 - val_mae: 0.5704 - val_mse: 0.4975
Epoch 3/200
230/230 [==============================] - 1s 5ms/step - loss: 0.5301 - mae: 0.5703 - mse: 0.5301 - val_loss: 0.4505 - val_mae: 0.5453 - val_mse: 0.4505
Epoch 4/200
230/230 [==============================] - 1s 4ms/step - loss: 0.5051 - mae: 0.5613 - mse: 0.5051 - val_loss: 0.4181 - val_mae: 0.5260 - val_mse: 0.4181
Epoch 5/200
230/230 [==============================] - 1s 4ms/step - loss: 0.4656 - mae: 0.5390 - mse: 0.4656 - val_loss: 0.4296 - val_mae: 0.5312 - val_mse: 0.4296
Epoch 6/200
230/230 [==============================] - 1s 4ms/step - loss: 0.4422 - mae: 0.5234 - mse: 0.4422 - val_loss: 0.3970 - val_mae: 0.5112 - val_mse: 0.3970
Epoch 7/20

In [42]:
history2 = model_wording.fit(features, target2, epochs=200,batch_size=25, validation_split=0.2)

Epoch 1/200
230/230 [==============================] - 3s 6ms/step - loss: 1.0344 - mae: 0.8057 - mse: 1.0344 - val_loss: 0.7855 - val_mae: 0.7221 - val_mse: 0.7855
Epoch 2/200
230/230 [==============================] - 1s 6ms/step - loss: 0.8683 - mae: 0.7361 - mse: 0.8683 - val_loss: 0.5999 - val_mae: 0.6323 - val_mse: 0.5999
Epoch 3/200
230/230 [==============================] - 1s 4ms/step - loss: 0.7662 - mae: 0.6955 - mse: 0.7662 - val_loss: 0.5619 - val_mae: 0.6075 - val_mse: 0.5619
Epoch 4/200
230/230 [==============================] - 1s 4ms/step - loss: 0.7180 - mae: 0.6728 - mse: 0.7180 - val_loss: 0.5860 - val_mae: 0.6235 - val_mse: 0.5860
Epoch 5/200
230/230 [==============================] - 1s 4ms/step - loss: 0.7016 - mae: 0.6641 - mse: 0.7016 - val_loss: 0.5877 - val_mae: 0.6198 - val_mse: 0.5877
Epoch 6/200
230/230 [==============================] - 1s 4ms/step - loss: 0.6783 - mae: 0.6546 - mse: 0.6783 - val_loss: 0.5475 - val_mae: 0.5993 - val_mse: 0.5475
Epoch 7/20

In [43]:
evaluate_on_train_content = model_content.evaluate(features, target1)
evaluate_on_train_wording = model_wording.evaluate(features, target2)

224/224 [==============================] - 1s 2ms/step - loss: 0.3051 - mae: 0.4242 - mse: 0.3051


In [44]:
print('evaluate_on_train_content', evaluate_on_train_content)
print('evaluate_on_train_wording',evaluate_on_train_wording)

evaluate_on_train_content [0.1626022905111313, 0.31251370906829834, 0.1626022905111313]
evaluate_on_train_wording [0.30505073070526123, 0.42420095205307007, 0.30505073070526123]


In [45]:
content_prediction = model_content.predict(features)
wording_prediction = model_wording.predict(features)

224/224 [==============================] - 0s 2ms/step


### Predict on test

In [46]:
test_pred_content = model_content.predict(features_for_test)
test_pred_wording = model_wording.predict(features_for_test)

1/1 [==============================] - 0s 20ms/step


## submission

In [47]:
test_pred_content = test_pred_content.reshape(-1)
test_pred_wording = test_pred_wording.reshape(-1)

In [48]:
submission = pd.DataFrame({
    'student_id' : test['student_id'],
    'content' : test_pred_content,
    'wording' : test_pred_wording
})

In [49]:
submission.to_csv('submission.csv', index=False)

In [50]:
submission.head()

,student_id,content,wording
0,000000ffffff,-1.452873,-1.457102
1,222222cccccc,-1.568339,-1.494185
2,111111eeeeee,-1.455856,-1.431564
3,333333dddddd,-1.543789,-1.507653
